In [1]:
import requests
from requests_oauthlib import OAuth1

auth = OAuth1("FD0A617EAD11499C97C806F06FB4CBCD", "B091220B9B5342A78D592E991C41322A", "AAA4E5969A3A450F978A9192A3C96C04", "BCC0C30CD7144713B83F10CA24AB58BC")

def fetch_bricklink_inventory():
    url = "https://api.bricklink.com/api/store/v1/inventories"
    response = requests.get(url, auth=auth)
    if response.status_code == 200:
        return response.json()['data']
    else:
        print(f"Error: {response.status_code}")
        return None


In [2]:
lots=fetch_bricklink_inventory()

len(lots)

total_quantity = sum(item["quantity"] for item in lots)

print(f"Total Quantity: {total_quantity}")

Total Quantity: 2472


In [3]:
def fetch_brickowl_inventory(api_key="8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d"):
    url = f"https://api.brickowl.com/v1/inventory/list?key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


In [4]:
def reconcile_inventory(bricklink_data, brickowl_data, PRIMARY_STORE="BrickLink"):
    reconciled = []

    # Build a mapping from inventory_id to item in BrickLink
    bricklink_items = {}
    for item in bricklink_data:
        inventory_id = item['inventory_id']
        bricklink_items[inventory_id] = item

    # Build a mapping from external_lot_ids to item in BrickOwl
    brickowl_items = {}
    for item in brickowl_data:
        external_ids = item.get('external_lot_ids', {})
        other_id = external_ids.get('other')
        if other_id:
            try:
                other_id_int = int(other_id)
                brickowl_items[other_id_int] = item
            except ValueError:
                pass  # Skip if 'other_id' is not a valid integer

    # Now compare items based on inventory_id
    all_inventory_ids = set(bricklink_items.keys()).union(set(brickowl_items.keys()))

    for inventory_id in all_inventory_ids:
        bricklink_item = bricklink_items.get(inventory_id)
        brickowl_item = brickowl_items.get(inventory_id)

        bricklink_qty = bricklink_item['quantity'] if bricklink_item else 0
        brickowl_qty = int(brickowl_item['qty']) if brickowl_item else 0

        if bricklink_qty != brickowl_qty:
            # Determine which platform needs to be updated
            if PRIMARY_STORE == "BrickLink":
                final_quantity = bricklink_qty
                platform_to_update = "BrickOwl"
            else:
                final_quantity = brickowl_qty
                platform_to_update = "BrickLink"

            reconciled.append({
                'inventory_id': inventory_id,
                'sku': bricklink_item['item']['no'] if bricklink_item else brickowl_item.get('ids', [{}])[0].get('id'),
                'color_id': bricklink_item['color_id'] if bricklink_item else None,
                'final_quantity': final_quantity,
                'platform_to_update': platform_to_update,
                'bricklink_quantity': bricklink_qty,
                'brickowl_quantity': brickowl_qty,
            })

    return reconciled


In [13]:
reconcile=reconcile_inventory(fetch_bricklink_inventory(),fetch_brickowl_inventory())


print(reconcile)

[{'inventory_id': 443460971, 'sku': '55706', 'color_id': 2, 'final_quantity': 15, 'platform_to_update': 'BrickOwl', 'bricklink_quantity': 15, 'brickowl_quantity': 0}, {'inventory_id': 443692804, 'sku': '1183-1', 'color_id': 0, 'final_quantity': 50, 'platform_to_update': 'BrickOwl', 'bricklink_quantity': 50, 'brickowl_quantity': 0}]


In [14]:
len(reconcile)
# reconcile

2

In [7]:
def update_brickowl_item(lot_id, updates):
    url = 'https://api.brickowl.com/v1/inventory/update'
    params = {
        'key': "8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d",
        'lot_id': lot_id,
        **updates
    }
    try:
        print(params)
        response = requests.post(url, params=params)
        print(response)
        response.raise_for_status()  # Raises HTTPError for bad responses
        data = response.json()
        if data.get('status') == 'success':
            print(f"BrickOwl item {lot_id} updated successfully.")
            return True
        else:
            print(f"BrickOwl API error for lot {lot_id}: {data}")
            return False
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for BrickOwl lot {lot_id}: {http_err}")
        return False
    except Exception as err:
        print(f"An error occurred for BrickOwl lot {lot_id}: {err}")
        return False


In [8]:
def update_bricklink_item(inventory_id, updates):
    url = f'https://api.bricklink.com/api/store/v1/inventories/{inventory_id}'
    headers = {'Content-Type': 'application/json'}
    try:
        # response = oauth_session.put(url, json=updates, headers=headers)
        response = requests.put(url, json=updates, auth=auth)
        if response.status_code == 204:
            print(f"BrickLink item {inventory_id} updated successfully.")
            return True
        else:
            print(f"BrickLink API error for inventory {inventory_id}: {response.status_code}, {response.text}")
            return False
    except Exception as err:
        print(f"An error occurred for BrickLink inventory {inventory_id}: {err}")
        return False

In [9]:
# Synchronize inventories based on the reconciled data
def synchronize_inventories(reconciled_data):
    for discrepancy in reconciled_data:
        inventory_id = discrepancy['inventory_id']
        sku = discrepancy['sku']
        color_id = discrepancy['color_id']
        final_quantity = discrepancy['final_quantity']
        platform_to_update = discrepancy['platform_to_update']
        bricklink_quantity = discrepancy['bricklink_quantity']
        brickowl_quantity = discrepancy['brickowl_quantity']

        if platform_to_update == 'BrickOwl':
            # Update BrickOwl
            lot_id = inventory_id
            updates = {
                'absolute_quantity': final_quantity,
            }
            success = update_brickowl_item( lot_id, updates)
            if not success:
                print(f"Failed to update BrickOwl item with lot_id {lot_id}")
        elif platform_to_update == 'BrickLink':
            # Update BrickLink
            updates = {
                'quantity': final_quantity,
            }
            success = update_bricklink_item( inventory_id, updates)
            if not success:
                print(f"Failed to update BrickLink item with inventory_id {inventory_id}")
        else:
            print(f"Unknown platform to update: {platform_to_update}")

In [15]:
synchronize_inventories(reconcile)

HTTP error occurred for BrickOwl lot 443460971: 403 Client Error: Forbidden for url: https://api.brickowl.com/v1/inventory/update?key=8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d&lot_id=443460971&absolute_quantity=15
Failed to update BrickOwl item with lot_id 443460971
HTTP error occurred for BrickOwl lot 443692804: 403 Client Error: Forbidden for url: https://api.brickowl.com/v1/inventory/update?key=8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d&lot_id=443692804&absolute_quantity=50
Failed to update BrickOwl item with lot_id 443692804


In [5]:
def update_bricklink_inventory(sku, quantity):
    url = f"https://api.bricklink.com/api/store/v1/inventories/{sku}"
    payload = {"quantity": quantity}
    response = requests.put(url, json=payload, auth=auth)
    if response.status_code == 200:
        print(f"Updated BrickLink inventory for {sku}")
    else:
        print(f"Error updating BrickLink inventory for {sku}")


In [6]:
def update_brickowl_inventory(api_key, sku, quantity):
    url = f"https://www.brickowl.com/api/inventory-update?apiKey={api_key}&sku={sku}&quantity={quantity}"
    response = requests.post(url)
    if response.status_code == 200:
        print(f"Updated Brick Owl inventory for {sku}")
    else:
        print(f"Error updating Brick Owl inventory for {sku}")


In [ ]:
from apscheduler.schedulers.background import BackgroundScheduler

scheduler = BackgroundScheduler()

def sync_job():
    bricklink_data = fetch_bricklink_inventory()
    brickowl_data = fetch_brickowl_inventory("8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d")
    if bricklink_data and brickowl_data:
        discrepancies = reconcile_inventory(bricklink_data, brickowl_data)
        print(discrepancies)
        for item in discrepancies:
            print(item)
            update_bricklink_inventory(item['sku'], item['brickowl_qty'])
            print("success")
            update_brickowl_inventory("8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d", item['sku'], item['brickowl_qty'])

scheduler.add_job(sync_job, 'interval', minutes=10)
scheduler.start()


In [12]:
def sync_job():
    bricklink_data = fetch_bricklink_inventory()
    print("1st")
    print(bricklink_data)
    brickowl_data = fetch_brickowl_inventory("8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d")
    print("2nd")
    print(brickowl_data)
    if bricklink_data and brickowl_data:
        discrepancies = reconcile_inventory(bricklink_data, brickowl_data)
        print(discrepancies)
        for item in discrepancies:
            print(item)
            update_bricklink_inventory(item['sku'], item['brickowl_qty'])
            print("success")
            update_brickowl_inventory("8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d", item['sku'], item['brickowl_qty'])
sync_job()

1st
[{'inventory_id': 440956339, 'item': {'no': '2420', 'name': 'Plate 2 x 2 Corner', 'type': 'PART', 'category_id': 26}, 'color_id': 48, 'color_name': 'Sand Green', 'quantity': 20, 'new_or_used': 'N', 'unit_price': '0.1600', 'bind_id': 0, 'description': 'Good Condition', 'remarks': '76154', 'bulk': 1, 'is_retain': True, 'is_stock_room': False, 'date_created': '2024-11-09T05:00:00.000Z', 'my_cost': '0.0000', 'sale_rate': -500, 'tier_quantity1': 0, 'tier_price1': '0.0000', 'tier_quantity2': 0, 'tier_price2': '0.0000', 'tier_quantity3': 0, 'tier_price3': '0.0000', 'my_weight': '0.0000'}, {'inventory_id': 440956297, 'item': {'no': '3020', 'name': 'Plate 2 x 4', 'type': 'PART', 'category_id': 26}, 'color_id': 7, 'color_name': 'Blue', 'quantity': 20, 'new_or_used': 'N', 'unit_price': '0.0600', 'bind_id': 0, 'description': 'Good Condition', 'remarks': '76154', 'bulk': 1, 'is_retain': True, 'is_stock_room': False, 'date_created': '2024-11-09T05:00:00.000Z', 'my_cost': '0.0000', 'sale_rate': -

In [13]:
import unittest

class TestSyncFunctions(unittest.TestCase):
    def test_reconcile_inventory(self):
        bricklink_data = [{'itemNo': '123', 'quantity': 5}]
        brickowl_data = [{'sku': '123', 'quantity': 3}]
        discrepancies = reconcile_inventory(bricklink_data, brickowl_data)
        self.assertEqual(len(discrepancies), 1)
        self.assertEqual(discrepancies[0]['brickowl_qty'], 3)



In [14]:
import unittest

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [8]:
def add_item_to_bricklink(item_no, color_id, quantity, unit_price):
    url = "https://api.bricklink.com/api/store/v1/inventories"
    payload = {
        "item": {
            "no": item_no,
            "type": "PART"  # Use SET for this item
        },
        "color_id": color_id,  # For sets, you might pass a default color like 0 (no specific color)
        "quantity": quantity,
        "new_or_used": "U",  # 'N' for New, 'U' for Used
        "completeness": "C",  # Use 'C' for Complete or 'I' for Incomplete
        "unit_price": f"{float(unit_price):.4f}",  # Ensure price is formatted as a string
        "bind_id": 0,
        "description": "",
        "remarks": "",
        "bulk": 1,
        "is_retain": False,
        "is_stock_room": False,
        "sale_rate": 0,
        "my_cost": "0.0000",
        "tier_quantity1": 0,
        "tier_price1": "0.0000",
        "tier_quantity2": 0,
        "tier_price2": "0.0000",
        "tier_quantity3": 0,
        "tier_price3": "0.0000"
    }
    response = requests.post(url, json=payload, auth=auth)
    if response.status_code == 200:
        print(f"Item {item_no} added to BrickLink successfully.Item details {response.json()}")
    else:
        print(f"Error adding item {item_no} to BrickLink: {response.status_code}, {response.json()}")

# Example Usage
add_item_to_bricklink("55706", 2, 15, 30.0)  # Add set 1435-1 with quantity 15 and price $10.0


Item 55706 added to BrickLink successfully.Item details {'meta': {'description': 'OK_CREATED', 'message': 'OK_CREATED', 'code': 201}, 'data': {'inventory_id': 443460971, 'item': {'no': '55706', 'name': 'Dragon Wing 8 x 10', 'type': 'PART', 'category_id': 148}, 'color_id': 2, 'quantity': 15, 'new_or_used': 'U', 'unit_price': '30.0000', 'bind_id': 0, 'remarks': '', 'bulk': 1, 'is_retain': False, 'is_stock_room': False, 'date_created': '2024-11-21T06:41:00.000Z', 'my_cost': '0.0000', 'sale_rate': 0, 'tier_quantity1': 0, 'tier_price1': '0.0000', 'tier_quantity2': 0, 'tier_price2': '0.0000', 'tier_quantity3': 0, 'tier_price3': '0.0000', 'my_weight': '0.0000'}}


In [39]:
inventory = fetch_bricklink_inventory()


In [40]:
new_item = next((item for item in inventory if item['inventory_id'] == 442270731), None)


In [41]:
print(new_item)

{'inventory_id': 442270731, 'item': {'no': '1435-1', 'name': 'Super Glider', 'type': 'SET', 'category_id': 516}, 'color_id': 2, 'color_name': 'Tan', 'quantity': 15, 'new_or_used': 'U', 'completeness': 'C', 'unit_price': '30.0000', 'bind_id': 0, 'remarks': '', 'bulk': 1, 'is_retain': False, 'is_stock_room': False, 'date_created': '2024-11-15T12:02:00.000Z', 'my_cost': '0.0000', 'sale_rate': 0, 'tier_quantity1': 0, 'tier_price1': '0.0000', 'tier_quantity2': 0, 'tier_price2': '0.0000', 'tier_quantity3': 0, 'tier_price3': '0.0000', 'my_weight': '0.0000'}


In [42]:
def update_item_on_bricklink(inventory_id, quantity, unit_price):
    url = f"https://api.bricklink.com/api/store/v1/inventories/{inventory_id}"
    payload = {
        "quantity": quantity,
        "unit_price": f"{float(unit_price):.4f}"
    }
    response = requests.put(url, json=payload, auth=auth)
    if response.status_code == 200:
        print(f"Item with inventory ID {inventory_id} updated successfully.")
    else:
        print(f"Error updating item {inventory_id}: {response.status_code}, {response.json()}")

# Example Usage
update_item_on_bricklink(442269142, 30, 25.0)  # Updates quantity to 10 and price to 15.00


Item with inventory ID 442269142 updated successfully.


In [43]:
inventory = fetch_bricklink_inventory()


In [44]:
new_item = next((item for item in inventory if item['inventory_id'] == 442269142), None)


In [45]:
print(new_item)

{'inventory_id': 442269142, 'item': {'no': '1435-1', 'name': 'Super Glider', 'type': 'SET', 'category_id': 516}, 'color_id': 0, 'color_name': '(Not Applicable)', 'quantity': 35, 'new_or_used': 'U', 'completeness': 'C', 'unit_price': '25.0000', 'bind_id': 0, 'remarks': '', 'bulk': 1, 'is_retain': False, 'is_stock_room': False, 'date_created': '2024-11-15T11:53:00.000Z', 'my_cost': '0.0000', 'sale_rate': 0, 'tier_quantity1': 0, 'tier_price1': '0.0000', 'tier_quantity2': 0, 'tier_price2': '0.0000', 'tier_quantity3': 0, 'tier_price3': '0.0000', 'my_weight': '0.0000'}


In [ ]:
# add_item_to_brickowl(new_item['item']['no'], new_item['quantity'], new_item['unit_price'])


In [ ]:
a=0o10
print(a)

10


In [7]:
def get_inventory_id(item_no):
    url = "https://api.bricklink.com/api/store/v1/inventories"
    response = requests.get(url, auth=auth)
    
    if response.status_code == 200:
        inventories = response.json()['data']
        for inventory in inventories:
            if inventory['item']['no'] == item_no:
                return inventory['inventory_id'], inventory
        return None, None  # If the item is not found
    else:
        print(f"Error retrieving inventory: {response.status_code}, {response.text}")
        return None, None

# Example Usage
inventory_id, inventory_details = get_inventory_id("1435-1")
if inventory_id:
    print(f"Inventory ID: {inventory_id}")
    print("Inventory Details:", inventory_details)
else:
    print("Item not found in inventory.")

get_inventory_id("1435-1")

Inventory ID: 442269142
Inventory Details: {'inventory_id': 442269142, 'item': {'no': '1435-1', 'name': 'Super Glider', 'type': 'SET', 'category_id': 516}, 'color_id': 0, 'color_name': '(Not Applicable)', 'quantity': 65, 'new_or_used': 'U', 'completeness': 'C', 'unit_price': '25.0000', 'bind_id': 0, 'remarks': '', 'bulk': 1, 'is_retain': False, 'is_stock_room': False, 'date_created': '2024-11-15T11:53:00.000Z', 'my_cost': '0.0000', 'sale_rate': 0, 'tier_quantity1': 0, 'tier_price1': '0.0000', 'tier_quantity2': 0, 'tier_price2': '0.0000', 'tier_quantity3': 0, 'tier_price3': '0.0000', 'my_weight': '0.0000'}


(442269142,
 {'inventory_id': 442269142,
  'item': {'no': '1435-1',
   'name': 'Super Glider',
   'type': 'SET',
   'category_id': 516},
  'color_id': 0,
  'color_name': '(Not Applicable)',
  'quantity': 65,
  'new_or_used': 'U',
  'completeness': 'C',
  'unit_price': '25.0000',
  'bind_id': 0,
  'remarks': '',
  'bulk': 1,
  'is_retain': False,
  'is_stock_room': False,
  'date_created': '2024-11-15T11:53:00.000Z',
  'my_cost': '0.0000',
  'sale_rate': 0,
  'tier_quantity1': 0,
  'tier_price1': '0.0000',
  'tier_quantity2': 0,
  'tier_price2': '0.0000',
  'tier_quantity3': 0,
  'tier_price3': '0.0000',
  'my_weight': '0.0000'})

In [12]:
def fetch_brickowl_orders():
    url = f"https://api.brickowl.com/v1/order/list?key=8245b56a4db48751882052166a763cdc617a532de0fab7c8f2521fc376de5b7d"

    try:
        response = requests.get(url)
        print(response.text)

        if response.status_code == 200:
            print("Orders fetched successfully.")
            return response.json()  # Returns the list of orders
        else:
            print(f"Error fetching orders: {response.status_code}, {response.text}")
            return []
    except requests.RequestException as e:
        print(f"Error: {e}")
        return []

# Example Usage
if __name__ == "__main__":
    orders = fetch_brickowl_orders()
    if orders:
        print(f"Fetched {len(orders)} orders.")
        print(orders)
    else:
        print("No orders fetched or an error occurred.")

[]
Orders fetched successfully.
No orders fetched or an error occurred.
